In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import statsmodels.api as sm
import shap
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from  sklearn.linear_model import Lasso
from sklearn.metrics import r2_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import KFold
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [85]:
import os
working_directory = os.getcwd()
print(working_directory)

/Users/ilinamitra


In [113]:
path = working_directory + '/Documents/df_longer.csv'
df = pd.read_csv(path, encoding='latin-1')
df.head()

,County,County.Code,suicides,Population,Crude_Rate,Uninsured: %,Mental_Health_Providers,Social_Associations,vet_pop_2010,Opioid Dispensing Rate per 100,...,in_labor_force,civilian_labot_force,employed,unemployed,armed_force,not_labor_force,total_occupied_housing,owner_occupied,renter_occupied,median_gross_rent
0,"Baldwin County, AL",1003.0,51,229287,22.242866,10.9,228.0,226.0,19614.0,65.0,...,99495.0,99317.0,95091.0,4226.0,178.0,72802.0,80930.0,60896.0,20034.0,1020.0
1,"Calhoun County, AL",1015.0,28,113469,24.676343,12.9,161.0,152.0,10000.0,97.9,...,52674.0,52106.0,47896.0,4210.0,568.0,40212.0,44605.0,31254.0,13351.0,716.0
2,"Etowah County, AL",1055.0,24,102371,23.444139,13.8,135.0,155.0,7232.0,102.8,...,45161.0,45149.0,42650.0,2499.0,12.0,38081.0,38942.0,28267.0,10675.0,669.0
3,"Houston County, AL",1069.0,16,106580,15.012197,13.6,170.0,124.0,9193.0,136.5,...,47603.0,47417.0,44374.0,3043.0,186.0,35593.0,39311.0,25829.0,13482.0,754.0
4,"Jefferson County, AL",1073.0,83,655342,12.665143,10.0,1379.0,953.0,42503.0,139.0,...,326672.0,326101.0,304994.0,21107.0,571.0,198562.0,261231.0,163799.0,97432.0,900.0


In [114]:
df = df.drop(["Population","County", "County.Code","Year", "suicides","StateAbbr", "StateDesc", "LocationName", 
              "State", "...1"],axis=1)

In [115]:
df.shape

(599, 103)

In [116]:
df.columns[df.isna().any()].tolist()

['Uninsured: %',
 'Mental_Health_Providers',
 'Social_Associations',
 'vet_pop_2010',
 'Opioid Dispensing Rate per 100',
 'homicide_rate',
 'firearm_death_rate',
 'RUCC_2013',
 'avg_temp',
 'precip',
 'Current lack of health insurance among adults aged 18-64 years',
 'Obesity among adults aged >=18 years',
 'Stroke among adults aged >=18 years',
 'Current asthma among adults aged >=18 years',
 'Diagnosed diabetes among adults aged >=18 years',
 'Depression among adults aged >=18 years',
 'Arthritis among adults aged >=18 years',
 'Chronic obstructive pulmonary disease among adults aged >=18 years',
 'Binge drinking among adults aged >=18 years',
 'Current smoking among adults aged >=18 years',
 'Physical health not good for >=14 days among adults aged >=18 years',
 'Coronary heart disease among adults aged >=18 years',
 'No leisure-time physical activity among adults aged >=18 years',
 'High blood pressure among adults aged >=18 years',
 'Mental health not good for >=14 days among adul

In [117]:
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

In [118]:
df_imputed = pd.DataFrame(imputer.fit_transform(df), columns=df.columns)
y = df_imputed['Crude_Rate']
X = df_imputed.loc[:, df_imputed.columns != 'Crude_Rate'] 

In [119]:
# VIF dataframe
vif_data = pd.DataFrame()
vif_data["feature"] = X.columns
  
# calculating VIF for each feature
#vif_data["VIF"] = [variance_inflation_factor(X.values, i)›
                          #for i in range(len(X.columns))]

vif_data["VIF"] = [variance_inflation_factor(df_imputed.loc[:, df_imputed.columns != 'Crude_Rate'].values, i) 
                   for i in range(len(df_imputed.loc[:, df_imputed.columns != 'Crude_Rate'].columns))]
vif_data

divide by zero encountered in double_scalars


,feature,VIF
0,Uninsured: %,7.445544e+01
1,Mental_Health_Providers,3.780193e+01
2,Social_Associations,1.057270e+02
3,vet_pop_2010,1.411636e+02
4,Opioid Dispensing Rate per 100,1.256958e+01
5,homicide_rate,1.130404e+02
6,firearm_death_rate,1.283268e+02
7,RUCC_2013,1.299855e+01
8,avg_temp,3.268907e+02
9,precip,2.254009e+01


In [120]:
#dropping extraneous and collinear
df_imputed = df_imputed.drop(["total_median_age", "female_median_age","total_population", "black_pop", "asian_pop", 
         "islander_pop", "other_pop", "mixed_pop", "family_household", "other_family", "single_man_family", 
         "single_woman_family", "nonfamily_household", "single_family_alone", "single_family_notalone",
         "nursery", "kindergarten", "grade_1", "grade_2", "grade_3", "grade_4", "grade_5", "grade_6", "grade_7",
         "grade_8", "grade_9", "grade_10", "grade_11", "grade_12_no_diploma", "hs_diploma", "GED", "some_college",
         "some_college_no_degree", "associates_degree", "bachelors_degree", "professional_degree", "doctoral_degree",
        "under_0.5_ip", "0.5_0.9_ip", "1_1.24_ip", "1.25_1.49_ip", "1.5_1.84_ip", "1.85_1.99_ip",
        "assistance_income", "no_assistance_income", "income_per_capita", "total_emp", "in_labor_force", 
         "civilian_labot_force", "employed", "armed_force", "not_labor_force", "owner_occupied", "renter_occupied",
          "dem_perc", "repub_perc",
             'Stroke among adults aged >=18 years', 
              'Physical health not good for >=14 days among adults aged >=18 years',
              'Coronary heart disease among adults aged >=18 years',
              'High blood pressure among adults aged >=18 years',
              'Mental health not good for >=14 days among adults aged >=18 years',
              'Fair or poor self-rated health status among adults aged >=18 years',
              'Cholesterol screening among adults aged >=18 years',
              'Chronic kidney disease among adults aged >=18 years',
              'Visits to doctor for routine checkup within the past year among adults aged >=18 years',
              'Cancer (excluding skin cancer) among adults aged >=18 years',
              'Taking medicine for high blood pressure control among adults aged >=18 years with high blood pressure',                   
            'High cholesterol among adults aged >=18 years who have been screened in the past 5 years',
              'Arthritis among adults aged >=18 years',
              'Current asthma among adults aged >=18 years',
              'Obesity among adults aged >=18 years',
              'Current lack of health insurance among adults aged 18-64 years',
              'Chronic obstructive pulmonary disease among adults aged >=18 years',
              'No leisure-time physical activity among adults aged >=18 years',
              'Diagnosed diabetes among adults aged >=18 years',
              'Current smoking among adults aged >=18 years',
              'total_public_assistance',
              'median_gross_rent', 'median_household_income',                
     'homicide_rate', 'firearm_death_rate','avg_temp', 
                              'Depression among adults aged >=18 years', 'Binge drinking among adults aged >=18 years',
               'over_2_ip', 'total_occupied_housing',
                              'married_couple_family', 'white_pop','male_median_age', 'Social_Associations',
                              'unemployed'
                             
                             ],axis=1)

In [121]:
y = df_imputed['Crude_Rate']
X = df_imputed.loc[:, df_imputed.columns != 'Crude_Rate'] 

vif_data = pd.DataFrame()
vif_data["feature"] = X.columns
  
# calculating VIF for each feature
#vif_data["VIF"] = [variance_inflation_factor(X.values, i)›
                          #for i in range(len(X.columns))]

vif_data["VIF"] = [variance_inflation_factor(df_imputed.loc[:, df_imputed.columns != 'Crude_Rate'].values, i) 
                   for i in range(len(df_imputed.loc[:, df_imputed.columns != 'Crude_Rate'].columns))]
vif_data

,feature,VIF
0,Uninsured: %,7.822703
1,Mental_Health_Providers,10.109481
2,vet_pop_2010,6.294137
3,Opioid Dispensing Rate per 100,7.511014
4,RUCC_2013,6.693159
5,precip,7.726082
6,Score,2.462720
7,Gun_Death_Rate_100k,12.093185
8,native_pop,2.184374
9,no_school,5.428338


In [95]:
df_imputed

,Crude_Rate,Uninsured: %,Mental_Health_Providers,vet_pop_2010,Opioid Dispensing Rate per 100,RUCC_2013,precip,Score,Gun_Death_Rate_100k,native_pop,no_school,masters_degree
0,22.242866,10.90000,228.000000,19614.000000,65.000000,3.000000,63.270000,-8.500000,23.600000,1645.000000,1179.000000,11812.000000
1,24.676343,12.90000,161.000000,10000.000000,97.900000,3.000000,68.910000,-8.500000,23.600000,379.000000,1053.000000,4484.000000
2,23.444139,13.80000,135.000000,7232.000000,102.800000,3.000000,71.270000,-8.500000,23.600000,581.000000,1033.000000,3444.000000
3,15.012197,13.60000,170.000000,9193.000000,136.500000,3.000000,67.270000,-8.500000,23.600000,271.000000,1031.000000,4385.000000
4,12.665143,10.00000,1379.000000,42503.000000,139.000000,1.000000,70.320000,-8.500000,23.600000,1443.000000,3681.000000,36620.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
594,13.955241,9.88398,1357.052721,24529.897133,49.077403,1.893761,43.891284,17.264334,14.370995,2804.780776,4577.677909,28385.838111
595,13.955241,9.88398,1357.052721,24529.897133,49.077403,1.893761,43.891284,17.264334,14.370995,2804.780776,4577.677909,28385.838111
596,13.955241,9.88398,1357.052721,24529.897133,49.077403,1.893761,43.891284,17.264334,14.370995,2804.780776,4577.677909,28385.838111
597,13.955241,9.88398,1357.052721,24529.897133,49.077403,1.893761,43.891284,17.264334,14.370995,2804.780776,4577.677909,28385.838111


In [96]:
y = df_imputed['Crude_Rate']
X = df_imputed.loc[:, df_imputed.columns != 'Crude_Rate'] 

def MLpipe(X,y,ML_algo,param_grid):
    '''
    This function splits the data to other/test (70/30)
    The R2 is maximized in cross-validation.
    '''
    
    random_state = 33
    
    # split the data
    #train on less? try 0.7 and then more in test set
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=random_state)

    std_scaler = StandardScaler()
    #std_fit = std_scaler.fit(X_train)
    X_train = std_scaler.fit_transform(X_train)
    #X_val = std_scaler.transform(X_val)
    X_test = std_scaler.transform(X_test)
    #print('mean_xtrain': np.mean()) #df.mean() 
        
    #kfold split
    kf = KFold(n_splits=10,shuffle=True,random_state=24)
        
    # loop through the hyperparameter combinations or use GridSearchCV 
    grid = GridSearchCV(estimator = ML_algo, param_grid=param_grid,scoring = 'neg_mean_absolute_error', 
                            n_jobs = -1, cv = kf, return_train_score = True, verbose=True) 
        
    #fitting on training data
    grid.fit(X_train, y_train)
    results = pd.DataFrame(grid.cv_results_)

    print('best model parameters:',grid.best_params_)
    print('validation score:',grid.best_score_) # this is the mean validation score over all iterations
    
    # calculate and save the test score
    y_test_pred = grid.predict(X_test)
    test_score = mean_absolute_error(y_test,y_test_pred)
    print(test_score)
        
    return grid, test_score

ML_algo = Lasso()
param_grid = {#'alpha':[0.01, 0.02,0.03,0.04,0.05,0.06,0.07,0.08,0.09, 0.10],
              #'alpha':[0.0000000001, 0.00000000129, 0.0000000167, 0.000000215, 0.00000278, 0.0000359,
                      #0.000464, 0.00599, 0.0774, 1],
             'alpha':[0.00599   , 0.01392444, 0.02185889, 0.02979333, 0.03772778,
                      0.04566222, 0.05359667, 0.06153111, 0.06946556, 0.0774],
             'max_iter': [100000, 1000000]}

model, score = MLpipe(X,y,ML_algo,param_grid)
print(score) 

Fitting 10 folds for each of 20 candidates, totalling 200 fits
best model parameters: {'alpha': 0.02979333, 'max_iter': 100000}
validation score: -3.5119279282441687
3.2995469118104843
3.2995469118104843


In [100]:
#having chosen the values from the param grid, running the model on 70% of data (training) and testing on 30% of data (test set)

y = df_imputed['Crude_Rate']
X = df_imputed.loc[:, df_imputed.columns != 'Crude_Rate'] 

def MLpipe(X,y,ML_algo,param_grid):
    '''
    This function splits the data to other/test (70/30)
    The R2 is maximized in cross-validation.
    '''
        
    random_state = 43

    # split the data
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=random_state)

    std_scaler = StandardScaler()
    #std_fit = std_scaler.fit(X_train)
    X_train = std_scaler.fit_transform(X_train)
    #X_val = std_scaler.transform(X_val)
    X_test = std_scaler.transform(X_test)


    model = ML_algo.fit(X_train, y_train)

    # calculate and save the test score
    y_test_pred = model.predict(X_test)
    test_score = mean_absolute_error(y_test,y_test_pred)
    print('test score:',test_score)
        
    return model, test_score

ML_algo = Lasso(alpha = 0.02185889, max_iter = 100000)

model, score = MLpipe(X,y,ML_algo,param_grid)
print(score) 

test score: 3.472130606478212
3.472130606478212


In [101]:
model.coef_ 

array([ 0.23679018,  0.90845609,  1.05375683, -0.        ,  1.17222219,
       -0.81035041, -0.36649389,  1.27681775,  1.15339107, -1.16755545,
       -2.32056892])

In [102]:
pd.Series(model.coef_, index = X.columns)

Uninsured: %                      0.236790
Mental_Health_Providers           0.908456
vet_pop_2010                      1.053757
Opioid Dispensing Rate per 100   -0.000000
RUCC_2013                         1.172222
precip                           -0.810350
Score                            -0.366494
Gun_Death_Rate_100k               1.276818
native_pop                        1.153391
no_school                        -1.167555
masters_degree                   -2.320569
dtype: float64

In [103]:
#dropping zero'd out 
df_imputed = df_imputed.drop(["Opioid Dispensing Rate per 100"],axis=1)

In [107]:
y = df_imputed['Crude_Rate']
X = df_imputed.loc[:, df_imputed.columns != 'Crude_Rate'] 

def MLpipe(X,y,ML_algo,param_grid):
    '''
    This function splits the data to other/test (70/30)
    The R2 is maximized in cross-validation.
    '''
        
    random_state = 43

    # split the data
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=random_state)

    std_scaler = StandardScaler()
    #std_fit = std_scaler.fit(X_train)
    X_train = std_scaler.fit_transform(X_train)
    #X_val = std_scaler.transform(X_val)
    X_test = std_scaler.transform(X_test)

    #Baseline linear regression model
    linreg = LinearRegression(fit_intercept=True)
    model = lingreg.fit(X_train, y_train)

    # calculate and save the test score
    y_test_pred = model.predict(X_test)
    test_score = mean_absolute_error(y_test,y_test_pred)
    print('test score:',test_score)
        
    return model, test_score
    
print(score)

3.472130606478212


In [112]:
# Add a constant term to the data for the intercept
df_imputed = sm.add_constant(df_imputed)

# Split the data into the dependent and independent variables
y = df_imputed['Crude_Rate']
X = df_imputed.loc[:, df_imputed.columns != 'Crude_Rate'] 

# Train the linear regression model
model = sm.OLS(y, X).fit()

# Extract the coefficients and p-values
coefs = model.params
p_values = model.pvalues

# Print the results
print("Coefficients:")
print(coefs)
print("\nP-Values:")
print(p_values)

Coefficients:
const                      10.278396
Uninsured: %                0.055696
Mental_Health_Providers     0.000638
vet_pop_2010                0.000028
RUCC_2013                   1.350175
precip                     -0.047930
Score                      -0.016939
Gun_Death_Rate_100k         0.276556
native_pop                  0.000152
no_school                  -0.000112
masters_degree             -0.000053
dtype: float64

P-Values:
const                      5.055362e-12
Uninsured: %               3.050096e-01
Mental_Health_Providers    1.007358e-03
vet_pop_2010               2.075845e-02
RUCC_2013                  2.592742e-09
precip                     1.208182e-05
Score                      4.065424e-01
Gun_Death_Rate_100k        1.474423e-07
native_pop                 1.520316e-05
no_school                  2.725598e-04
masters_degree             3.948017e-07
dtype: float64
